## Excluding subgraphs from backward
每个变量(Variable)都有两个标签：requires_grad和volatile。通过这两个标签中的任何一个，我们都可以控制，在后向梯度运算的时候，部分子图不会被计算到，从而提高整体运算的效率。
#### reguires_grad
假设存在一个运算Operation，那么他的输出使不需要梯度的，当且仅当该运算的所有输入都是不需要梯度的。当一个子图中的所有变量都是不需要梯度的，那么这个子图在后向传播计算梯度的时候，不会被计算到。

In [5]:
import torch
from torch.autograd import Variable

x = Variable(torch.rand(5, 5))
y = Variable(torch.rand(5, 5))
z = Variable(torch.rand(5, 5), requires_grad = True)
a = x + y
print(a.requires_grad)
b = a + z
print(b.requires_grad)

False
True


这种方法在想冻结部分模型的时候相当有用<br>
For example if you want to finetune a pretrained CNN, it's enouhh to switch the requires_grad flags in the frozen base, and no intermediate buffers will be saved, until the computations gets to the last layer, where the affine transform will use weights that require gradient, and the output of the network will also require them.<br>

In [10]:
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
model = resnet18(pretrained = True)
for param in model.parameters():
    param.requires_grad = False
# Replace the last fully connected layer
# Parameters of newly constructed modules have requires_grad = True by default
model.fc = nn.Linear(512, 100)
# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr = 1e-2, momentum = 0.9)
# 不是都关了吗，把整个model的参数都放进去还有差别吗？？？？？？？

#### volatile
Volatile一般推荐在推理模式中(inference model)使用，也就是说当你很确定你不会用.backward()的时候。他比其他的autograd设置，更加有效。同时volatile也意味着require_grad的值是False。
Volatile和require_grad的区别主要在于他们传播的方式。假设我们有一个运算Operation，那么当其中存在一个input是Volatile的时候，对应的输出也就是Volatile的。Volatile传播得更快，因为对于一个子图，你只需要设置他起初的一个input为Volatile的就行了，而使用require_grad，你需要把其中每个变量都设一遍。

In [14]:
require_input = Variable(torch.randn(1, 3, 227, 227))
volatile_input = Variable(torch.randn(1, 3, 227, 227), volatile = True)
model = resnet18(pretrained = True)
model(require_input).requires_grad

True

In [15]:
model(volatile_input).requires_grad

False

In [16]:
model(volatile_input).volatile

True

In [17]:
model(volatile_input).grad_fn is None

True

## How autograd encoded the history
Autograd is reverse automatic differentiation system.不知道怎么翻译？？？<br>
从概念上来讲，autograd记录着一个无环图，在这个图中记载着用于产生对应输出的所有运算过程，他的叶子节点代表输入变量，根节点代表输出。通过链式规则，我们由根节点到各个叶子节点算出其中各个节点的梯度。<br>
从内部实现来说，Autograd描述的是一个由function组成的无向图。(which can be apply()-ed to cimpute the result of evaluating the graph). 当执行Forward运算的时候，autograd会同时处理要求的所有计算，并构建一个用于计算梯度的无向图。当前向传播搞定之后，他通过这个新构建的图，计算各个节点的梯度。<br>
一个比较重要的点是，在循环中的每次运算都会重新生成一个图。你完全可以在这期间，通过控制流，改变图的结构和大小。也就是说，每次循环跑的图，可以是不一样的。同样的，你也不需要等到实现了所有可能的路径之后，再进行训练。<br>
An important thing to note is that the graph is recreated from scratch at evary iteration, and this is exactly what allows for using arbitrary Python control flow statements, that can change  the overall shape and size of the graph at every iteration. You don't have to encode all possible paths before you launch the training - what you run is what you differentialte.

## In-place operations on Variables
